
### **(KR)**

- 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
- LLM은 **항상** 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
- 예제를 제공하려면 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하세요.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# chat model
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
examples = [
    {
        "movie": "아바타1",
        "answer": """
        I know this:
        director: 제임스 카메론
        main cast:샘 워딩턴,조 샐다나,스티븐 랭,미셸 로드리게스,시고니 위버,
        budget: 2억 3700만 달러
        revenue: 29억 2,370만 6,026달러
        genre: SF
        synopsis: 인류의 마지막 희망, 행성 판도라!
        이곳을 정복하기 위한 ‘아바타 프로젝트’가 시작된다!
        가까운 미래, 지구는 에너지 고갈 문제를 해결하기 위해 머나먼 행성 판도라에서 대체 자원을 채굴하기 시작한다. 하지만 판도라의 독성을 지닌 대기로 인해 자원 획득에 어려움을 겪게 된 인류는 판도라의 토착민 나비족(Na'vi)의 외형에 인간의 의식을 주입, 원격 조종이 가능한 새로운 생명체 ‘아바타’를 탄생시키는 프로그램을 개발한다.

        평범한 삶을 살아가던 전직 해병대원 제이크,
        우주의 미래가 걸린 거대한 운명이 그에게 찾아왔다!
        한편, 하반신이 마비된 전직 해병대원 ‘제이크 설리(샘 워싱턴)’는 ‘아바타 프로그램’에 참가할 것을 제안 받아 판도라로 향한다. 그곳에서 자신의 ‘아바타’를 통해 자유롭게 걸을 수 있게 된 ‘제이크’는 자원 채굴을 막으려는 나비족(Na'vi)의 무리에 침투하라는 임무를 부여 받는다. 임무 수행 중 나비족(Na'vi)의 여전사 ‘네이티리(조 샐다나)’를 만난 ‘제이크’는 그녀와 함께 다채로운 모험을 경험하면서 ‘네이티리’를 사랑하게 되고, 그들과 하나가 되어간다. 하지만 머지않아 전 우주의 운명을 결정짓는 대규모 전투가 시작되면서 ‘제이크’는 최후의 시험대에 오르게 되는데….

        행성 판도라와 지구의 피할 수 없는 전쟁!
        이 모든 운명을 손에 쥔 제이크의 선택은?

    """,
    },
    {
        "movie": "어벤져스",
        "answer": """
        I know this:
        director: 조스 웨던
        main cast: 로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요한슨, 제러미 레너, 톰 히들스턴, 새뮤얼 L. 잭슨
        budget: 2억 2,000만 달러
        revenue: 15억 1,881만 5,515달러
        genre: 슈퍼 히어로
        synopsis: ASSEMBLE! 최강의 슈퍼히어로들이 모였다!지구의 운명을 건 거대한 전쟁이 시작된다!지구의 안보가 위협당하는 위기의 상황에서 슈퍼히어로들을 불러모아 세상을 구하는, 일명 [어벤져스] 작전. 에너지원 ‘큐브’를 이용한 적의 등장으로 인류가 위험에 처하자 국제평화유지기구인 쉴드 (S.H.I.E.L.D)의 국장 닉 퓨리(사무엘 L.잭슨)는 [어벤져스] 작전을 위해 전 세계에 흩어져 있던 슈퍼히어로들을 찾아나선다. 아이언맨(로버트 다우니 주니어)부터 토르(크리스 헴스워스), 헐크(마크 러팔로), 캡틴 아메리카(크리스 에반스)는 물론, 쉴드의 요원인 블랙 위도우(스칼렛 요한슨), 호크 아이(제러미 레너)까지, 최고의 슈퍼히어로들이 [어벤져스]의 멤버로 모이게 되지만, 각기 개성이 강한 이들의 만남은 예상치 못한 방향으로 흘러가는데… 지구의 운명을 건 거대한 전쟁 앞에 [어벤져스] 작전은 성공할 수 있을까?


    """,
    },
    {
        "movie": "스파이더맨: 홈커밍",
        "answer": """
        I know this:
        director: 존 와츠
        main cast: 톰 홀랜드, 마이클 키턴, 젠데이아, 도널드 글러버, 제이컵 배털론, 로라 해리어, 토니 레볼로리, 타인 데일리, 보킴 우드바인, 머리사 토메이, 로버트 다우니 주니어
        budget: 1억 7500만 달러[
        revenue: $874,407,031
        genre: SF, 슈퍼히어로
        synopsis: ‘시빌 워’ 당시 ‘토니 스타크’(로버트 다우니 주니어)에게 발탁되어 대단한 활약을 펼쳤던 스파이더맨 ‘피터 파커’(톰 홀랜드). 그에게 새로운 수트를 선물한 ‘토니 스타크’는 위험한 일은 하지 말라며 조언한다. 하지만 허세와 정의감으로 똘똘 뭉친 ‘피터 파커’는 세상을 위협하는 강력한 적 ‘벌처’(마이클 키튼)에 맞서려 하는데… 아직은 어벤져스가 될 수 없는 스파이더맨 숙제보다 세상을 구하고 싶은 스파이더맨 그는 과연 진정한 히어로로 거듭날 수 있을 것인가!
    """,
    },
]

In [3]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

# Fewshot
few_shot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# final_prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a great movie reviewer."),
        few_shot_chat_prompt,
        ("human", "what do you know about {movie}?"),
    ]
)

In [4]:
chain = final_prompt | chat

chain.invoke({"movie": "어벤져스: 엔드 게임"})

I know this:
        director: 조 루소, 안소니 루소
        main cast: 로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요한슨, 제러미 레너, 폴 러드, 조슈 브롤린, 브리 라슨, 카렌 길런, 댄나 구리라, 베네딕트 컴버배치, 톰 홀랜드, 찰리스 샤프, 돈 치들, 브래들리 쿠퍼, 엠마 톰슨, 마크 퍼로스, 린다 카델리니, 로버트 레드포드, 조나스 브라더스
        budget: 3억 5,000만 달러
        revenue: 27억 8,000만 달러
        genre: 슈퍼히어로, SF
        synopsis: 인피니티 스톤을 손에 넣은 타노스(조슈 브롤린)는 우주의 절반을 살해하고, 지구의 어벤져스는 타노스에게 패배한다. 그 후 5년이 흐른 지구는 절망에 빠져 있고, 어벤져스 멤버들도 각자의 삶을 살아가고 있다. 그러던 중, 스콧 랭(폴 러드)이 양자 영역에서 탈출하며 희망의 빛이 되고, 어벤져스는 다시 모여 타노스에 맞서기로 결심한다. 어벤져스의 최후의 전투가 시작되는데, 이번에는 모든 것이 끝나야만 새로운 시작이 될 것이다. 어벤져스와 타노스의 최후의 전투, 그리고 엔드 게임이 시작된다.

AIMessageChunk(content='I know this:\n        director: 조 루소, 안소니 루소\n        main cast: 로버트 다우니 주니어, 크리스 에반스, 마크 러팔로, 크리스 헴스워스, 스칼렛 요한슨, 제러미 레너, 폴 러드, 조슈 브롤린, 브리 라슨, 카렌 길런, 댄나 구리라, 베네딕트 컴버배치, 톰 홀랜드, 찰리스 샤프, 돈 치들, 브래들리 쿠퍼, 엠마 톰슨, 마크 퍼로스, 린다 카델리니, 로버트 레드포드, 조나스 브라더스\n        budget: 3억 5,000만 달러\n        revenue: 27억 8,000만 달러\n        genre: 슈퍼히어로, SF\n        synopsis: 인피니티 스톤을 손에 넣은 타노스(조슈 브롤린)는 우주의 절반을 살해하고, 지구의 어벤져스는 타노스에게 패배한다. 그 후 5년이 흐른 지구는 절망에 빠져 있고, 어벤져스 멤버들도 각자의 삶을 살아가고 있다. 그러던 중, 스콧 랭(폴 러드)이 양자 영역에서 탈출하며 희망의 빛이 되고, 어벤져스는 다시 모여 타노스에 맞서기로 결심한다. 어벤져스의 최후의 전투가 시작되는데, 이번에는 모든 것이 끝나야만 새로운 시작이 될 것이다. 어벤져스와 타노스의 최후의 전투, 그리고 엔드 게임이 시작된다.')